In [ ]:
# Save this as app.py
from flask import Flask, render_template, request, redirect, url_for, flash
import os
import tensorflow as tf
import cv2
import numpy as np
from tqdm import tqdm
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
import joblib

# Create a new model for feature extraction (use the second-to-last layer of InceptionV3)
def create_feature_extraction_model():
    base_model = InceptionV3(include_top=False, pooling='avg')
    feature_model = Model(inputs=base_model.input, outputs=base_model.output)
    return feature_model

# Function to extract frames and obtain their features
def extract_features_from_video(video_path, model, sequence_length=16, img_size=(299, 299)):
    # Initialize the list for storing features
    video_features = []
    
    # Open the video file
    video_reader = cv2.VideoCapture(video_path)
    frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(int(frame_count / sequence_length), 1)

    for frame_counter in tqdm(range(sequence_length), desc="Extracting frames"):
        # Set the frame position and read it
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * frame_interval)
        success, frame = video_reader.read()
        if not success:
            break
        
        # Pre-process the frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized_frame = cv2.resize(frame_rgb, img_size)
        normalized_frame = resized_frame / 255.0
        
        # Extract features using the model
        features = extract_frame_features(normalized_frame, model)
        
        # Append the features to the list
        video_features.append(features)
    
    # Release the video reader
    video_reader.release()
    
    return np.array(video_features)

# Feature extraction function
def extract_frame_features(frame, model):
    img_expanded = np.expand_dims(frame, axis=0)
    feature_vector = model.predict(img_expanded, verbose=0)
    return feature_vector

loaded_model = tf.keras.models.load_model("violence.h5")
app = Flask(__name__)
app.secret_key = 'your_secret_key'  # For session and flash messages

# Directory to save uploaded videos
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
      if request.method == 'POST':
        # Check if the 'xrayImage' file is in the request
        if 'video' in request.files:
            # Get the uploaded file
            uploaded_file = request.files['video']

            # Check if the file has a valid filename
            if uploaded_file.filename:
                # Save the uploaded file to a specific directory
                uploaded_file.save(r'C:\Users\shiva\major\uploads\ ' + uploaded_file.filename)
                # Generate the URL for the uploaded image
                video_path = r"C:\Users\shiva\major\uploads\ " + uploaded_file.filename  # Replace with the path to your image
                feature_model = create_feature_extraction_model()
                
                # Extract features from the video
                features = extract_features_from_video(video_path, feature_model, sequence_length=16)
                feat=features.reshape(1,16,2048)
                prediction = loaded_model.predict(feat)
                if prediction>0.25:
                    prediction="Non Violence"
                else:
                    prediction="Violence"
              
        return render_template("index.html",prediction=prediction)
    
if __name__ == '__main__':
    app.run(port=4000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4000
Press CTRL+C to quit
127.0.0.1 - - [28/Apr/2024 00:05:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2024 00:05:33] "GET /favicon.ico HTTP/1.1" 404 -
Extracting frames: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  4.79it/s]


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [28/Apr/2024 00:07:28] "POST /upload HTTP/1.1" 200 -
Extracting frames: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:08<00:00,  1.76it/s]


1/1 [==============================] - 0s 36ms/step


127.0.0.1 - - [28/Apr/2024 00:08:07] "POST /upload HTTP/1.1" 200 -
Extracting frames: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  4.91it/s]


1/1 [==============================] - 0s 36ms/step


127.0.0.1 - - [28/Apr/2024 00:08:22] "POST /upload HTTP/1.1" 200 -
